## Computing values on hybrid levels

In [1]:
import earthkit.meteo.vertical.array as vertical

This notebook gives an overview about computing various quantities on hybrid levels.

#### Hybrid levels

Hybrid levels divide the atmosphere into layers. These layers are defined by the pressures at the interfaces between them, which are the **half levels**. The half levels are defined by a set A and B coefficients in such a way that at the top of the atmosphere the first half level pressure is a constant, while at the bottom the last one is the surface pressure. The **full level** pressure associated with each model level is defined as the middle of the layer. As a consequence, the bottom-most (full) model level is always above the surface. Please note that by convention the model level numbering starts at 1 at the top of the atmosphere and increases towards the surface.

For more details about the hybrid levels see [IFS Documentation CY47R3 - Part IV Physical processes, Chapter 2, Section 2.2.1.](https://www.ecmwf.int/en/elibrary/20198-ifs-documentation-cy47r3-part-iv-physical-processes)

#### Getting the data

First, we get some sample data containing 137 IFS (full) model levels for two points. This data is defined on full model levels in ascending model level number order. So the first level is model level 1, while the last one is model level 137.

In [2]:
from earthkit.meteo.utils.sample import get_sample

DATA = get_sample("vertical_hybrid_data")
sp = DATA.p_surf # surface pressure [Pa]
zs = DATA.z_surf # surface geopotential [m2/s2]
t = DATA.t # temperature [K]
q = DATA.q # specific humidity [kg/kg]

sp.shape, zs.shape, t.shape, q.shape

((2,), (2,), (137, 2), (137, 2))

In [3]:
A, B = vertical.hybrid_level_parameters(137, model="ifs")
A.shape, B.shape

((138,), (138,))

The A and B arrays contain coefficients for each half hybrid level in ascending model level number order.

#### Computing pressure

In [4]:
p_full = vertical.pressure_on_hybrid_levels(A, B, sp, output="full")
p_full.shape

(137, 2)

The output is ordered by ascending model level number, so goes from the top of the atmosphere towards the surface.

In [5]:
# the lowest 3 levels above the surface
p_full[-3:]

array([[ 94572.33402543, 102005.12627431],
       [ 94829.60969192, 102283.5225447 ],
       [ 95065.55727665, 102538.16442829]])

By using the ``output`` kwarg other pressure related quantities can be computed.

In [6]:
p_full, p_half, alpha,delta = vertical.pressure_on_hybrid_levels(
    A, B, sp, 
    output=("full", "half","alpha", "delta"))

p_full.shape, p_half.shape, alpha.shape, delta.shape

((137, 2), (138, 2), (137, 2), (137, 2))

It is possible to use only part of the levels via the ``levels`` option. The level indices start at 1 and go up to 137 for the current data. The example below only computes the pressure on the lowest 3 levels.

In [7]:
p_full = vertical.pressure_on_hybrid_levels(
    A, B, sp, levels=[135, 136, 137],
    output=("full"))
p_full

array([[ 94572.33402543, 102005.12627431],
       [ 94829.60969192, 102283.5225447 ],
       [ 95065.55727665, 102538.16442829]])

The ``level`` kwarg respects the specified ordering. In the example below the output goes upwards from the lowest level.

In [8]:
p_full = vertical.pressure_on_hybrid_levels(
    A, B, sp, levels=[137, 136, 135],
    output=("full"))
p_full

array([[ 95065.55727665, 102538.16442829],
       [ 94829.60969192, 102283.5225447 ],
       [ 94572.33402543, 102005.12627431]])

#### Computing geopotential thickness

In [9]:
z_thickness= vertical.relative_geopotential_thickness_on_hybrid_levels(t, q, 
    A, B, sp)

Note that ``t`` and ``q`` must contain the same model levels in ascending order with respect to the model level number. The model level range must be contiguous and must include the bottom-most level, but not all the levels must be present. E.g. for our data we can compute the relative geopotential thickness for the lowest 3 levels as shown below. Please note that even in this case we use the full A and B arrays.

In [10]:
z_thickness= vertical.relative_geopotential_thickness_on_hybrid_levels(t[-3:], q[-3:], A, B, sp)
z_thickness

array([[493.46642507, 540.17339501],
       [283.62079769, 310.18495411],
       [ 91.62754053, 100.19126806]])

#### Computing geopotential

In [11]:
z = vertical.geopotential_on_hybrid_levels(t, q, zs, A, B, sp)
z.shape, z[-3:]

((137, 2),
 array([[5774.59166921, -289.70136085],
        [5564.74604183, -519.68980175],
        [5372.75278467, -729.68348779]]))

Just like for the relative geopotential thickness a subset of levels can be used. E.g. to compute the geopotential for the lowest 3 levels we can write:

In [12]:
z = vertical.geopotential_on_hybrid_levels(t[-3:], q[-3:], zs, A, B, sp)
z.shape, z

((3, 2),
 array([[5774.59166921, -289.70136085],
        [5564.74604183, -519.68980175],
        [5372.75278467, -729.68348779]]))

#### Computing height

In [13]:
h = vertical.height_on_hybrid_levels(t, q, zs, A, B, sp, h_type="geometric", h_reference="ground")
h.shape, h[-3:]

((137, 2),
 array([[50.3284765 , 55.08136987],
        [28.92629362, 31.62937948],
        [ 9.34500226, 10.21641103]]))

In [14]:
h = vertical.height_on_hybrid_levels(t, q, zs, A, B, sp, h_type="geometric", h_reference="sea")
h.shape, h[-3:]

((137, 2),
 array([[588.89890231, -29.54118048],
        [567.49671944, -52.99317088],
        [547.91542808, -74.40613933]]))

Just like for the previous methods a subset of levels can be used. E.g. to compute the geometric height above ground for the lowest 3 levels we can write:

In [15]:
h = vertical.height_on_hybrid_levels(t[-3:], q[-3:], 0, A, B, sp, h_type="geometric", h_reference="ground")
h.shape, h

((3, 2),
 array([[50.31996884, 55.08283309],
        [28.92140386, 31.6302197 ],
        [ 9.34342257, 10.21668243]]))